# Preprocessing - Youtube Videos

### Objectives

Our objective is: dslkjaldfgjsdf

### Data Description

dkfgkdjhskgh

In [1]:
import pyspark
sc.version

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1587565607163_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2.4.4'

In [2]:
#Read and load csv file
df = (
spark
.read
.csv(
#'s3://cis4567-winniechan/Spark/Dataset/YoutubeData.csv'
's3://cis4567-apinchapong/Final_Project/YoutubeData.csv'
, header=True
, inferSchema=True)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------+-------------------------------------+--------------------------+------------+-----------+------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+--------------------+-------------+
|   video_id|trending_date|                                title|             channel_title|publish_date|category_id|publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|       category_name|       region|
+-----------+-------------+-------------------------------------+--------------------------+------------+-----------+------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+--------------------+-------------+
|hLqHZGhIFaI|   2017-12-2

In [4]:
#Count the number of rows
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

416869

In [5]:
#show the number of columns
len(df.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

19

In [6]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['video_id', 'trending_date', 'title', 'channel_title', 'publish_date', 'category_id', 'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'description', 'category_name', 'region']

### Finding Duplicates

In [7]:
#This shows the number of rows v the number of distinct rows
df.count(), df.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(416869, 368752)

In [8]:
#Dropping the duplicated rows
no_duplicates= df.dropDuplicates()
no_duplicates.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

368752

#### Removing duplicates where only video_id differs

In [9]:
#Compare no_duplicates with different_ids, where the row has identical data besides video_id
different_ids = (
    no_duplicates
    .select([col for col in no_duplicates.columns if col != 'video_id'])
)

different_ids.count(), different_ids.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(368752, 364423)

In [10]:
#remove the duplicated records
no_duplicated_rows = no_duplicates.dropDuplicates(
    subset = [col for col in no_duplicates.columns if col != 'video_id']
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#count the number of rows for no_duplicated_rows
no_duplicated_rows.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

364423

In [12]:
#no_duplicated_rows.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# (
#     no_duplicated_rows
#     .where('video_id == IThIVh9MpQw')
#     .show()
# )
# #Doch wichtig ist

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#commented out
# are there any duplicated IDs?
""""import pyspark.sql.functions as fn

no_duplicated_rows.agg(
      fn.count('video_id').alias('CountOfIDs')
    , fn.countDistinct('video_id').alias('CountOfDistinctIDs')
).show() """

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'"import pyspark.sql.functions as fn\n\nno_duplicated_rows.agg(\n      fn.count(\'video_id\').alias(\'CountOfIDs\')\n    , fn.countDistinct(\'video_id\').alias(\'CountOfDistinctIDs\')\n).show() '

### Handling Missing Values

#### Missing Observations in Rows

In [15]:
#examine if there are any rows with missing values
no_duplicated_rows.rdd.map(
           lambda row: (
                 row['video_id']
               , sum([c == None for c in row])
           )
        )\
        .filter(lambda y: y[1] >= 1)\
        .count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

27109

In [16]:
#drop rows that have less than thresh NON-NULL values. 
#drop values that have less than 10 non-null values
df_row_threshold = no_duplicated_rows.dropna(thresh=10)
no_duplicated_rows.count(), df_row_threshold.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(364423, 363068)

#### Missing Observations per column

In [18]:
import pyspark.sql.functions as fn

#count percentage of missing values in each column
for k, v in sorted(
    no_duplicated_rows.agg(*[
               (1 - (fn.count(c) / fn.count('*')))
                    .alias(c + '_miss')
               for c in no_duplicated_rows.columns
           ])
        .collect()[0]
        .asDict()
        .items()
    , key=lambda el: el[1]
    , reverse=True
):
    print(k, v)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

description_miss 0.055155684465579835
category_name_miss 0.023930981304692622
region_miss 0.01652200876454013
video_error_or_removed_miss 0.003871874168205669
ratings_disabled_miss 0.0038636419764943586
comments_disabled_miss 0.003847177593071738
thumbnail_link_miss 0.0038252250818416877
comment_count_miss 0.0038005285067078676
dislikes_miss 0.0037758319315740474
likes_miss 0.0037182065895950966
views_miss 0.003644116864193525
tags_miss 0.0030047499746174156
publish_time_miss 0.0027934570540278925
category_id_miss 0.0022940374235435046
publish_date_miss 0.0019620056911885397
channel_title_miss 0.001487282635837972
title_miss 0.0008396835545506631
trending_date_miss 5.488127807540266e-06
video_id_miss 0.0

In [19]:
#Column null percentage is low no need to remove

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# #Possible change to 1???

# df_row_threshold = no_duplicated_rows.dropna(thresh=10)
# no_duplicated_rows.count(), df_row_threshold.count()